## Imports

In [17]:
import random
import torch
from torch.autograd import Variable
from tensorboard import SummaryWriter

## Dynamic Net Model

In [18]:
class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


## Model parameters

In [19]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10


## Instatiating tensorboard writer

In [20]:
writer = SummaryWriter()

In [21]:

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    writer.add_scalar('loss', loss.data.numpy()[0], t)
    print(t, loss.data[0])
    if t % 10 == 0:
        for name, param in model.named_parameters():
            writer.add_histogram(name, param.clone().data.numpy(), t)


    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
z = model(Variable(torch.randn(N, D_in), requires_grad=True))
writer.add_graph(model, z)
writer.close()

(0, 677.1868286132812)
(1, 678.1227416992188)
(2, 676.561767578125)
(3, 698.6341552734375)
(4, 671.9229125976562)
(5, 670.2515869140625)
(6, 573.4230346679688)
(7, 652.8018188476562)
(8, 467.53564453125)
(9, 409.5158386230469)
(10, 623.6585693359375)
(11, 301.4396057128906)
(12, 659.091552734375)
(13, 655.9367065429688)
(14, 583.3419189453125)
(15, 656.2310791015625)
(16, 645.7720336914062)
(17, 530.07861328125)
(18, 140.73728942871094)
(19, 645.3768920898438)
(20, 451.0494689941406)
(21, 102.75556182861328)
(22, 583.298095703125)
(23, 564.9273071289062)
(24, 539.439208984375)
(25, 589.3126220703125)
(26, 470.9310607910156)
(27, 130.41708374023438)
(28, 260.17230224609375)
(29, 237.04893493652344)
(30, 347.32928466796875)
(31, 193.80763244628906)
(32, 171.730224609375)
(33, 257.2962951660156)
(34, 332.11798095703125)
(35, 197.78152465820312)
(36, 140.16236877441406)
(37, 228.80674743652344)
(38, 134.16421508789062)
(39, 215.24569702148438)
(40, 188.85292053222656)
(41, 156.294464111328

## To see the tensorboard results do ` tensorboard --logdir runs`